In [13]:
import numpy as np
import pandas as pd

import os

import transformers
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import LabelEncoder

from transformers import AutoTokenizer, BertModel, DistilBertModel
from transformers import AutoModel, BertForSequenceClassification, BertTokenizer

from datasets import Dataset, ClassLabel

import torch
from torch.utils.data import Dataset as TorchDataset, DataLoader
from torch.nn import TripletMarginLoss
from torch.optim import Adam
from tqdm import tqdm

import neptune.new as neptune

In [14]:
dataset_df = pd.read_csv('dataset/embeddings_smol.csv')

In [15]:
class TweetDataset(TorchDataset):
    def __init__(self, dataset: pd.DataFrame):
        self.embeddings = torch.Tensor(dataset.drop(['labels']))
        self.labels = torch.Tensor(dataset['labels'])

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, item):
        anchor = self.embeddings[item]
        anchor_class = self.labels[item]

        positive_indices = self.labels == anchor_class
        positive_indices = positive_indices.nonzero()
        positive_idx = positive_indices[torch.randint(high=len(positive_indices), size=(1, ))[0]]
        #positive_example = self.input_ids[positive_idx].flatten()
        #positive_attention = self.attention_mask[positive_idx]
        positive_example = self.embeddings[positive_idx]

        negative_indices = self.labels != anchor_class
        negative_indices = negative_indices.nonzero()
        negative_idx = negative_indices[torch.randint(high=len(negative_indices), size=(1, ))[0]]
        #negative_example = self.input_ids[negative_idx].flatten()
        #negative_attention = self.attention_mask[negative_idx]
        negative_example = self.embeddings[negative_idx]

        return anchor, positive_example, negative_example

In [12]:
dataset = TweetDataset(dataset_df)

KeyError: "['labels'] not found in axis"